In [1]:
import gurobipy as gb
from gurobipy import *
import numpy as np

# Rummikub

Rummikub is a game that combines elements of classic card games like Rummy with the strategy of tile placement. The game is played with a set of 106 tiles, with numbers ranging from 1 to 13 in four different colors (red, blue, yellow, and black). Additionally, there are two joker tiles in the set. The game is typically played by 2 to 4 players and revolves around the strategic placement and manipulation of numbered tiles. The objective of Rummikub is to be the first player to empty your rack of tiles by forming sets and runs of matching numbers. Sets consist of three or four tiles of the same number but different colors. For example, you could have a set of 3s with one red, one blue, and one black. Runs are sequences of at least three consecutive numbers of the same color. For instance, you could have a run of 4, 5, 6 in blue. The game continues until one player goes out, at which point they gain opponents' tile values, while others receive penalties determined by the remaining tiles in their racks.

In this project, our primary objective is to address Rummikub challenges through the application of integer linear programming. Initially, we plan to focus on a two-player scenario, with the potential to expand to a four-player format if time permits. Also, if time allows, our ultimate goal is to develop an interactive online Rummikub board game, providing users with a platform for engaging gameplay.

## Set up
$2*4*13 + joker*2$

In [2]:
model = gb.Model("Rummikub")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-30


#### All tiles

In [3]:
Deck_types = ["Black", "Red", "Orange", "Blue"]
Deck_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
Joker = "Joker"
Joker_values = 30

In [4]:
tiles_pool = [(color, value) for color in Deck_types for value in Deck_values]
tiles_pool.append((Joker, Joker_values))

In [5]:
len(tiles_pool)

53

In [6]:
tiles_pool

[('Black', 1),
 ('Black', 2),
 ('Black', 3),
 ('Black', 4),
 ('Black', 5),
 ('Black', 6),
 ('Black', 7),
 ('Black', 8),
 ('Black', 9),
 ('Black', 10),
 ('Black', 11),
 ('Black', 12),
 ('Black', 13),
 ('Red', 1),
 ('Red', 2),
 ('Red', 3),
 ('Red', 4),
 ('Red', 5),
 ('Red', 6),
 ('Red', 7),
 ('Red', 8),
 ('Red', 9),
 ('Red', 10),
 ('Red', 11),
 ('Red', 12),
 ('Red', 13),
 ('Orange', 1),
 ('Orange', 2),
 ('Orange', 3),
 ('Orange', 4),
 ('Orange', 5),
 ('Orange', 6),
 ('Orange', 7),
 ('Orange', 8),
 ('Orange', 9),
 ('Orange', 10),
 ('Orange', 11),
 ('Orange', 12),
 ('Orange', 13),
 ('Blue', 1),
 ('Blue', 2),
 ('Blue', 3),
 ('Blue', 4),
 ('Blue', 5),
 ('Blue', 6),
 ('Blue', 7),
 ('Blue', 8),
 ('Blue', 9),
 ('Blue', 10),
 ('Blue', 11),
 ('Blue', 12),
 ('Blue', 13),
 ('Joker', 30)]

#### All Possible Sets

In [7]:
# Adjusting the setup to consider only the deck color
Deck_colors = ["Black", "Red", "Blue", "Orange"]

# Generating all possible sets with three consecutive numbers
# and represent each card in the set using Deck_color and Deck_values
three_consecutive_set_no_joker = []

for value in Deck_values[:-2]:  # Iterate through the values, stopping two before the end
    for color in Deck_colors:
        # Create a set of three consecutive cards of the same color
        consecutive_set = [(color, value), (color, value + 1), (color, value + 2)]
        three_consecutive_set_no_joker.append(consecutive_set)


In [8]:
# Adjusting the setup to include one Joker in each set
# The Joker can replace any one of the three cards in the set

three_consecutive_set_one_joker = []
# Iterate through each color and value, creating sets with one Joker
for color in Deck_colors:
    for value in Deck_values[:-2]:  # Iterate through the values, stopping two before the end
        # For each set of three consecutive values, create three sets, each with one Joker
        if value == 1:
            for i in range(3):
                consecutive_set_one_joker = [(color, value), (color, value + 1), (color, value + 2)]
                consecutive_set_one_joker[i] = "Joker"  # Replace one card with a Joker
                three_consecutive_set_one_joker.append(consecutive_set_one_joker)
        else:
            for i in range(2):
                consecutive_set_one_joker = [(color, value), (color, value + 1), (color, value + 2)]
                consecutive_set_one_joker[i] = "Joker"  # Replace one card with a Joker
                three_consecutive_set_one_joker.append(consecutive_set_one_joker)


In [9]:
# Adjusting the setup to include exactly two Jokers in each set
# Each set will now consist of one card from the deck and two Jokers

# Generating all possible sets with one card from the deck and two Jokers
three_consecutive_sets_with_two_jokers = []

for color in Deck_colors:
    for value in Deck_values:  # Iterate through all values
        # Create a set with one card from the deck and two Jokers
        set_with_two_jokers = [(color, value), "Joker", "Joker"]
        three_consecutive_sets_with_two_jokers.append(set_with_two_jokers)

In [10]:
# Generating sets with four consecutive numbers, same color, and without Joker

four_consecutive_sets_no_joker = []

for color in Deck_colors:
    for value in Deck_values[:-3]:  # Iterate through values, stopping three before the end
        # Create a set of four consecutive cards of the same color
        consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
        four_consecutive_sets_no_joker.append(consecutive_set)

In [11]:
four_consecutive_sets_one_joker = []

for color in Deck_colors:
    for value in Deck_values[:-3]:  # Iterate through values, stopping three before the end
        # Create a set of four consecutive cards of the same color
        if value == 1:
            for i in range(4):
                consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
                consecutive_set[i] = "Joker"
                four_consecutive_sets_one_joker.append(consecutive_set)
        else:
            for i in range(3):
                consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
                consecutive_set[i] = "Joker"
                four_consecutive_sets_one_joker.append(consecutive_set)

In [12]:
four_consecutive_sets_two_joker = []

for color in Deck_colors:
    for value in Deck_values[:-3]:  # Iterate through values, stopping three before the end
        # Create a set of four consecutive cards of the same color
        if value == 1:
            for i in range(3):
                for j in range(i+1, 4):
                    consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
                    consecutive_set[i] = "Joker"
                    consecutive_set[j] = "Joker"
                    four_consecutive_sets_two_joker.append(consecutive_set)
        else:
            for i in range(2):
                for j in range(i+1, 3):
                    consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
                    consecutive_set[i] = "Joker"
                    consecutive_set[j] = "Joker"
                    four_consecutive_sets_two_joker.append(consecutive_set)

In [13]:
# Generating sets with five consecutive numbers, same color, and without Joker

five_consecutive_sets_no_joker = []

for color in Deck_colors:
    for value in Deck_values[:-4]:  # Iterate through values, stopping four before the end
        # Create a set of five consecutive cards of the same color
        consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
        five_consecutive_sets_no_joker.append(consecutive_set)

In [14]:
# Generating sets with five consecutive numbers, same color, and 1 Joker

five_consecutive_sets_one_joker = []

for color in Deck_colors:
    for value in Deck_values[:-4]:  # Iterate through values, stopping four before the end
        # Create a set of five consecutive cards of the same color
        if value == 1:
            for i in range(5):
                consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
                consecutive_set[i] = "Joker"
                five_consecutive_sets_one_joker.append(consecutive_set)
        else:
            for i in range(4):
                consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
                consecutive_set[i] = "Joker"
                five_consecutive_sets_one_joker.append(consecutive_set)

In [15]:
# Generating sets with five consecutive numbers, same color, and 2 Joker

five_consecutive_sets_two_joker = []

for color in Deck_colors:
    for value in Deck_values[:-4]:  # Iterate through values, stopping four before the end
        # Create a set of five consecutive cards of the same color
        if value == 1:
            for i in range(4):
                for j in range(i+1, 5):
                    consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
                    consecutive_set[i] = "Joker"
                    consecutive_set[j] = "Joker"
                    five_consecutive_sets_two_joker.append(consecutive_set)
        else:
            for i in range(3):
                for j in range(i+1, 4):
                    consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
                    consecutive_set[i] = "Joker"
                    consecutive_set[j] = "Joker"
                    five_consecutive_sets_two_joker.append(consecutive_set)

In [16]:
# Generating sets of three cards, each from a different color, without Jokers, and all cards having the same value

three_cards_different_colors_same_value = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    for i in range(len(Deck_colors)):
        for j in range(i + 1, len(Deck_colors)):
            for k in range(j + 1, len(Deck_colors)):
                # Create a set with one card from each of the three different colors, all with the same value
                set_of_three = [(Deck_colors[i], value), (Deck_colors[j], value), (Deck_colors[k], value)]
                three_cards_different_colors_same_value.append(set_of_three)

In [17]:
# Generating sets of three cards, each from a different color, 1 Jokers, and all cards having the same value

three_cards_different_colors_1_joker = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    for i in range(len(Deck_colors)):
        for j in range(i + 1, len(Deck_colors)):
            # Create a set with one card from each of the three different colors, all with the same value
            set_of_three = [(Deck_colors[i], value), (Deck_colors[j], value), "Joker"]
            three_cards_different_colors_1_joker.append(set_of_three)

In [18]:
four_cards_different_colors_same_value = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    set_of_four = [(Deck_colors[0], value), (Deck_colors[1], value), (Deck_colors[2], value), (Deck_colors[3], value)]
    four_cards_different_colors_same_value.append(set_of_four)

In [19]:
# Generating sets of four cards, each from a different color, 1 Jokers, and all cards having the same value

four_cards_different_colors_1_joker = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    for i in range(len(Deck_colors)):
        for j in range(i + 1, len(Deck_colors)):
            for k in range(j + 1, len(Deck_colors)):
                # Create a set with one card from each of the three different colors, all with the same value
                set_of_four = [(Deck_colors[i], value), (Deck_colors[j], value), (Deck_colors[k], value), "Joker"]
                four_cards_different_colors_1_joker.append(set_of_four)

In [20]:
# Generating sets of four cards, each from a different color, 2 Jokers, and all cards having the same value

four_cards_different_colors_2_joker = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    for i in range(len(Deck_colors)):
        for j in range(i + 1, len(Deck_colors)):
            # Create a set with one card from each of the three different colors, all with the same value
            set_of_four = [(Deck_colors[i], value), (Deck_colors[j], value), "Joker", "Joker"]
            four_cards_different_colors_2_joker.append(set_of_four)

In [21]:
three_consecutives = three_consecutive_set_no_joker + three_consecutive_set_one_joker + three_consecutive_sets_with_two_jokers
four_consecutives = four_consecutive_sets_no_joker + four_consecutive_sets_one_joker + four_consecutive_sets_two_joker
five_consecutives = five_consecutive_sets_no_joker + five_consecutive_sets_one_joker + five_consecutive_sets_two_joker
three_same = three_cards_different_colors_same_value + three_cards_different_colors_1_joker
four_same = four_cards_different_colors_same_value + four_cards_different_colors_1_joker + four_cards_different_colors_2_joker
all_sets = three_consecutives + four_consecutives + five_consecutives + three_same + four_same

In [22]:
len(all_sets)

1173

In [23]:
I = len(tiles_pool)
J = len(all_sets)

## Decision variables

In [24]:
S = model.addVars(I, J, vtype = GRB.BINARY, 
                  name = ["tile " + str(tiles_pool[i]) + " is in set " + str(all_sets[j]) 
                          for i in range(I) for j in range(J)])

In [25]:
T = model.addVars(I, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["number of tile " + str(tiles_pool[i]) + " is on the table" for i in range(I)])

In [26]:
R = model.addVars(I, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["number of tile " + str(tiles_pool[i]) + " is in the rack" for i in range(I)])

In [27]:
X = model.addVars(J, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["set " + str(all_sets[j]) + " can be placed onto the table" for j in range(J)])

In [28]:
Y = model.addVars(I, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["tile " + str(tiles_pool[i]) + " can be placed from your rack onto the table" for i in range(I)])

## Objective function

In [29]:
score = quicksum(Y[i] * tiles_pool[i][1] for i in range(I))
model.setObjective(score, GRB.MAXIMIZE)
#model.setObjective(quicksum(Y[i] for i in range(I)), GRB.MAXIMIZE)

## Constraints

In [30]:
for i in range(I):
    sx_sum = quicksum(S[i, j] * X[j] for j in range(J))
    model.addConstr(sx_sum == T[i] + Y[i])

In [31]:
for i in range(I):
    model.addConstr(Y[i] <= R[i])

### The real game

In [32]:
Rack = [("Orange", 1), ("Orange", 3), ("Orange", 4), ("Orange", 8), 
        ("Blue", 2), ("Black", 2), ("Black", 10), ("Black", 13), ("Joker", 30)]

Set = [[("Black", 1), ("Black", 2), ("Black", 3), ("Black", 4)], 
       [("Black", 10), ("Black", 11), ("Black", 12)],
       [("Orange", 6), ("Orange", 7), ("Orange", 8)], 
       [("Orange", 9), ("Orange", 10), ("Orange", 11), ("Orange", 12), ("Orange", 13)], 
       [("Blue", 7), ("Blue", 8), ("Blue", 9)], 
       [("Blue", 10), ("Blue", 11), ("Blue", 12)], 
       [("Red", 11), ("Red", 12), ("Red", 13)]]

Table = [card for set_ in Set for card in set_]

In [33]:
played_tiles = Rack + Table

In [34]:
Counter_all_played_tiles = {}
for tile in Rack:
    if tile not in Counter_all_played_tiles.keys():
        Counter_all_played_tiles[tile] = 1
    else:
        Counter_all_played_tiles[tile] += 1

for tile in Table:
    if tile not in Counter_all_played_tiles.keys():
        Counter_all_played_tiles[tile] = 1
    else:
        Counter_all_played_tiles[tile] += 1

In [35]:
Counter_all_played_tiles

{('Orange', 1): 1,
 ('Orange', 3): 1,
 ('Orange', 4): 1,
 ('Orange', 8): 2,
 ('Blue', 2): 1,
 ('Black', 2): 2,
 ('Black', 10): 2,
 ('Black', 13): 1,
 ('Joker', 30): 1,
 ('Black', 1): 1,
 ('Black', 3): 1,
 ('Black', 4): 1,
 ('Black', 11): 1,
 ('Black', 12): 1,
 ('Orange', 6): 1,
 ('Orange', 7): 1,
 ('Orange', 9): 1,
 ('Orange', 10): 1,
 ('Orange', 11): 1,
 ('Orange', 12): 1,
 ('Orange', 13): 1,
 ('Blue', 7): 1,
 ('Blue', 8): 1,
 ('Blue', 9): 1,
 ('Blue', 10): 1,
 ('Blue', 11): 1,
 ('Blue', 12): 1,
 ('Red', 11): 1,
 ('Red', 12): 1,
 ('Red', 13): 1}

In [36]:
Counter_rack = {}
for tile in Rack:
    if tile not in Counter_rack.keys():
        Counter_rack[tile] = 1
    else:
        Counter_rack += 1

In [37]:
Counter_rack

{('Orange', 1): 1,
 ('Orange', 3): 1,
 ('Orange', 4): 1,
 ('Orange', 8): 1,
 ('Blue', 2): 1,
 ('Black', 2): 1,
 ('Black', 10): 1,
 ('Black', 13): 1,
 ('Joker', 30): 1}

In [38]:
Counter_table = {}
for tile in Table:
    if tile not in Counter_table.keys():
        Counter_table[tile] = 1
    else:
        Counter_table[tile] += 1

In [39]:
Counter_table

{('Black', 1): 1,
 ('Black', 2): 1,
 ('Black', 3): 1,
 ('Black', 4): 1,
 ('Black', 10): 1,
 ('Black', 11): 1,
 ('Black', 12): 1,
 ('Orange', 6): 1,
 ('Orange', 7): 1,
 ('Orange', 8): 1,
 ('Orange', 9): 1,
 ('Orange', 10): 1,
 ('Orange', 11): 1,
 ('Orange', 12): 1,
 ('Orange', 13): 1,
 ('Blue', 7): 1,
 ('Blue', 8): 1,
 ('Blue', 9): 1,
 ('Blue', 10): 1,
 ('Blue', 11): 1,
 ('Blue', 12): 1,
 ('Red', 11): 1,
 ('Red', 12): 1,
 ('Red', 13): 1}

In [40]:
# Rack
for tile in Rack:
    i = tiles_pool.index(tile)
    
    # update T[i]
    if tile in Counter_table.keys(): 
        num_table = Counter_table[tile]
        model.addConstr(T[i] == num_table)
    else:
        model.addConstr(T[i] == 0)
        
    # update R[i]
    num_rack = Counter_rack[tile]
    model.addConstr(R[i] == num_rack)
    
    # update S[i, j] and X[j]
    for set_ in all_sets:
        j = all_sets.index(set_)
        if tile not in set_:
            #model.addConstr(X[j] == 0)
            model.addConstr(S[i, j] == 0)

In [41]:
# Table
for tile in Table:
    i = tiles_pool.index(tile)
    
    # update R[i]
    if tile in Counter_rack.keys(): 
        num_rack = Counter_rack[tile]
        model.addConstr(R[i] == num_rack)
    else:
        model.addConstr(R[i] == 0)
        
    # update T[i]
    num_table = Counter_table[tile]
    model.addConstr(T[i] == num_table)
    
    # update S[i, j] and X[j]
    for set_ in all_sets:
        j = all_sets.index(set_)
        if tile not in set_:
            #model.addConstr(X[j] == 0)
            model.addConstr(S[i, j] == 0)

In [42]:
# Account for Joker
num_joker = Counter_all_played_tiles[("Joker", 30)]
# i = 52 
for set_ in all_sets:
        j = all_sets.index(set_)
        if set_.count("Joker") == 2 and num_joker == 1:
            for i in range(I):
                model.addConstr(S[i, j] == 0)

In [43]:
not_in_rack_table = []
for tile in tiles_pool:
    if tile not in Rack and tile not in Table:
        not_in_rack_table.append(tile)

In [44]:
for tile in not_in_rack_table:
    i = tiles_pool.index(tile)
    model.addConstr(T[i] == 0)
    model.addConstr(R[i] == 0)
    for set_ in all_sets:
        if tile in set_:
            j = all_sets.index(set_)
            model.addConstr(X[j] == 0)
            model.addConstr(S[i, j] == 0)

## Optimize

In [45]:
# model.params.BestObjStop = 331

In [46]:
model.optimize()
model.status

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 66247 rows, 63501 columns and 66300 nonzeros
Model fingerprint: 0xf9f526b4
Model has 53 quadratic constraints
Variable types: 0 continuous, 63501 integer (62169 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 66247 rows and 62766 columns
Presolve time: 0.07s
Presolved: 1658 rows, 1278 columns, 4352 nonzeros
Variable types: 0 continuous, 1278 integer (551 binary)
Found heuristic solution: objective 43.0000000

Explored 1 nodes (0 simplex iterations) in 0.09 seconds (0.07 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 43 

Optimal solution fo

2

In [47]:
model.ObjVal

43.0

In [48]:
for j in range(J):
    if X[j].x != 0:
        print(X[j].varName, X[j].x)

set [('Black', 1), ('Black', 2), ('Black', 3)] can be placed onto the table 2.0
set [('Orange', 6), ('Orange', 7), ('Orange', 8)] can be placed onto the table 2.0
set [('Blue', 7), ('Blue', 8), ('Blue', 9)] can be placed onto the table 1.0
set [('Blue', 8), ('Blue', 9), ('Blue', 10)] can be placed onto the table 1.0
set [('Blue', 9), ('Blue', 10), ('Blue', 11)] can be placed onto the table 1.0
set [('Orange', 9), ('Orange', 10), ('Orange', 11)] can be placed onto the table 1.0
set [('Black', 10), ('Black', 11), ('Black', 12)] can be placed onto the table 2.0
set [('Blue', 10), ('Blue', 11), ('Blue', 12)] can be placed onto the table 1.0
set [('Orange', 10), ('Orange', 11), ('Orange', 12)] can be placed onto the table 1.0
set [('Black', 11), ('Black', 12), ('Black', 13)] can be placed onto the table 1.0
set [('Red', 11), ('Red', 12), ('Red', 13)] can be placed onto the table 1.0
set [('Orange', 11), ('Orange', 12), ('Orange', 13)] can be placed onto the table 1.0
set [('Black', 1), 'Jok

In [49]:
for i in range(I):
     if Y[i].x != 0:
        print(Y[i].varName, Y[i].x)

tile ('Black', 2) can be placed from your rack onto the table 1.0
tile ('Black', 10) can be placed from your rack onto the table 1.0
tile ('Black', 13) can be placed from your rack onto the table 1.0
tile ('Orange', 1) can be placed from your rack onto the table 1.0
tile ('Orange', 3) can be placed from your rack onto the table 1.0
tile ('Orange', 4) can be placed from your rack onto the table 1.0
tile ('Orange', 8) can be placed from your rack onto the table 1.0
tile ('Blue', 2) can be placed from your rack onto the table 1.0


In [50]:
for i in range(I):
    for j in range(J):
        if S[i,j].x == 1:
            print(S[i,j].varName, S[i,j].x)

tile ('Black', 1) is in set [('Black', 1), 'Joker', ('Black', 3)] 1.0
tile ('Black', 2) is in set [('Black', 1), ('Black', 2), ('Black', 3)] 1.0
tile ('Black', 2) is in set [('Black', 2), ('Black', 3), ('Black', 4)] 1.0
tile ('Black', 2) is in set ['Joker', ('Black', 2), ('Black', 3)] 1.0
tile ('Black', 2) is in set [('Black', 1), ('Black', 2), 'Joker'] 1.0
tile ('Black', 2) is in set [('Black', 2), 'Joker', ('Black', 4)] 1.0
tile ('Black', 2) is in set [('Black', 1), ('Black', 2), ('Black', 3), ('Black', 4)] 1.0
tile ('Black', 2) is in set ['Joker', ('Black', 2), ('Black', 3), ('Black', 4)] 1.0
tile ('Black', 2) is in set [('Black', 1), ('Black', 2), 'Joker', ('Black', 4)] 1.0
tile ('Black', 2) is in set [('Black', 1), ('Black', 2), ('Black', 3), 'Joker'] 1.0
tile ('Black', 2) is in set [('Black', 1), ('Black', 2), ('Black', 3), ('Black', 4), 'Joker'] 1.0
tile ('Black', 3) is in set [('Black', 1), 'Joker', ('Black', 3)] 1.0
tile ('Black', 4) is in set ['Joker', ('Black', 3), ('Black',

In [51]:
for i in range(I):
    for j in range(J):
        if S[i,j].x != 0 and X[j].x != 0:
              #print(S[i,j].varName, S[i,j].x)
            print(X[j].varName, X[j].x)

set [('Black', 1), 'Joker', ('Black', 3)] can be placed onto the table 1.0
set [('Black', 1), ('Black', 2), ('Black', 3)] can be placed onto the table 2.0
set [('Black', 1), 'Joker', ('Black', 3)] can be placed onto the table 1.0
set ['Joker', ('Black', 3), ('Black', 4)] can be placed onto the table 1.0
set [('Black', 10), ('Black', 11), ('Black', 12)] can be placed onto the table 2.0
set [('Black', 11), ('Black', 12), ('Black', 13)] can be placed onto the table 1.0
set [('Black', 11), ('Black', 12), ('Black', 13)] can be placed onto the table 1.0
set [('Black', 11), ('Black', 12), ('Black', 13)] can be placed onto the table 1.0
set [('Red', 11), ('Red', 12), ('Red', 13)] can be placed onto the table 1.0
set [('Red', 11), ('Red', 12), ('Red', 13)] can be placed onto the table 1.0
set [('Red', 11), ('Red', 12), ('Red', 13)] can be placed onto the table 1.0
set [('Orange', 1), 'Joker', ('Orange', 3)] can be placed onto the table 1.0
set ['Joker', ('Orange', 3), ('Orange', 4)] can be plac